In [1]:
import numpy as np
import itertools
from SALib.test_functions import Ishigami

def ishigami_wrapper(param_values):
    strip_dummy_param = param_values[:, 0:3]
    return Ishigami.evaluate(strip_dummy_param)

In [2]:
problem_with_dummy = {
  'num_vars': 4,
  'names': ['x1', 'x2', 'x3', 'x4'],
  'bounds': [[-np.pi, np.pi]]*4,
  'nominal': [1.0,] * 4
    
}

In [16]:
def oat_sample(problem):
    bounds = problem.get('bounds')
    nominal = problem.get('nominal')
    num_vars = problem.get('num_vars')
    
    # get nominal
    experiment = [nominal[:]]
    
    for i in range(num_vars):
        min_val = bounds[i][0]
        max_val = bounds[i][1]
        
        if i == 0:
            experiment.append([min_val] + nominal[i+1:])  # min
            experiment.append([max_val] + nominal[i+1:])  # max
        else:
            experiment.append(nominal[0:i] + [min_val] + nominal[i+1:])  # min
            experiment.append(nominal[0:i] + [max_val] + nominal[i+1:])  # max
    
    return np.array(experiment)
    
    
def oat_analyze(problem, inputs, results):
    X = inputs
    Y = results
    
    num_factors = inputs.shape[1]
    num_results = Y.size  # skip the first one as it is the nominal value
    nominal_values = problem.get('nominal')

    # Set up result array
    si_res = np.full(((Y.size - 1),), fill_value=np.nan)
    
    count = 0
    for i in range(num_factors):
        for h in range(1, num_results):
            if X[h][i] == nominal_values[i]:
                continue

            si_res[count] = (Y[h] - Y[0]) / (X[h][i] - X[0][i])
            count += 1
        # End for
    # End for
    
    return si_res
    

In [17]:
%%time
samples = oat_sample(problem_with_dummy)

Wall time: 0 ns


In [19]:
%time
Y = ishigami_wrapper(samples)

Wall time: 0 ns


In [20]:
%time
oat_analyze(problem_with_dummy, samples, Y)

array([ 0.22349327, -0.43221015,  1.19676519, -2.31440555, -1.95879845,
        3.7880898 , -0.        ,  0.        ])